In [72]:
import tensorflow as tf
import numpy as np
from preprocessed_mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()


In [73]:
X_train = np.reshape(X_train, (-1, 28*28))
X_val = np.reshape(X_val, (-1, 28*28))
X_test = np.reshape(X_test, (-1, 28*28))

In [77]:
import random

def add_layer(inputs, in_size, out_size, activation_function=None):
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0.0, stddev=0.1), name='W')
            tf.summary.histogram('weights', Weights)
        with tf.name_scope('bias'):
            bias = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram('bias', bias)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b=tf.matmul(inputs, Weights) + bias
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram('outputs', outputs)
        return outputs

def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy)
    return result

xs = tf.placeholder(tf.float32, [None, 784])    #28*28
ys = tf.placeholder(tf.float32, [None, 10])

layer_1 = add_layer(xs, 784, 100, activation_function=tf.nn.relu)
prediction = add_layer(layer_1, 100, 10, activation_function=tf.nn.softmax)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction), axis=1))
tf.summary.scalar('xentropy_loss', cross_entropy)

#default lr for Adam is 0.001, this is smaller tham that of GD
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
merge = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)
sess.run(init)



In [75]:
y_train = sess.run(tf.one_hot(tf.squeeze(y_train), depth=10))
y_val = sess.run(tf.one_hot(tf.squeeze(y_val), depth=10))
y_test = sess.run(tf.one_hot(tf.squeeze(y_test), depth=10))

In [78]:
for i in range(100005):
    batch = random.sample(range(X_train.shape[0]), 100)
    sess.run(train_step, feed_dict={xs: X_train[batch], ys: y_train[batch]})
    if i%500==0:
       print (compute_accuracy(X_val, y_val))

0.1113
0.8673
0.8985
0.9118
0.9182
0.9245
0.93
0.9345
0.9386
0.9419
0.9435
0.946
0.948
0.9469
0.9504
0.9522
0.9533
0.9538
0.9556
0.9569
0.9578
0.9584
0.9602
0.96
0.961
0.9621
0.9617
0.963
0.9637
0.9645
0.9649
0.9653
0.9654
0.9659
0.9668
0.9666
0.9669
0.9669
0.9676
0.9675
0.9678
0.9681
0.9685
0.969
0.9695
0.97
0.9697
0.9697
0.9702
0.9699
0.9694
0.9704
0.971
0.9698
0.9708
0.9705
0.9712
0.9713
0.9713
0.9702
0.9708
0.9707


KeyboardInterrupt: 